In [24]:
%pip install requests beautifulsoup4 pandas numpy selenium

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.parse import urlparse, parse_qs, unquote
import asyncio
import pandas as pd

def saveToCSV(data: list, filename: str):
  df = pd.DataFrame(data, columns=["Facebook Links"])
  df.to_csv(filename, index=False)


async def getDriver(driver, facebook_page):
  driver.get(f"https://www.facebook.com/{facebook_page}")
  await asyncio.sleep(0.5)

def filter_Button(x):
  if x.get_attribute("aria-label") == "Close" and x.get_attribute("role") == "button" and str(x.get_attribute("tabindex")) == "0":
    return True
  return False

def get_facebook_links(facebook_link: str, facebook_page: str):
  if facebook_link.startswith(f"https://l.facebook.com/l.php"):
    parsed_url = urlparse(facebook_link)
    query_params = parse_qs(parsed_url.query)
    url = query_params.get("u", [None])
    if len(url) > 0:
      return unquote(url[0])
  elif facebook_link.startswith(f"https://www.facebook.com/{facebook_page}/") or facebook_link.startswith(f"https://www.facebook.com/photo/"):
    return facebook_link
  return None

async def main():
  driver = webdriver.Chrome()
  fb_page = "fopsco.ph"
  await getDriver(driver, fb_page)
  buttonToClose = driver.find_elements(By.TAG_NAME, "div")
  # "x1i10hfl xjqpnuy xa49m3k xqeqjp1 x2hbi6w x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi x1ypdohk xdl72j9 x2lah0s xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r x2lwn1j xeuugli x16tdsg8 x1hl2dhg xggy1nq x1ja2u2z x1t137rt x1q0g3np x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xzolkzo x12go9s9 x1rnf11y xprq8jg x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x78zum5 xl56j7k xexx8yu x4uap5 x18d9i69 xkhd6sd x1n2onr6 xc9qbxq x14qfxbe x1qhmfi1"
  filteredButton = list(filter(filter_Button, buttonToClose))
  for x in filteredButton:
    x.click()
    await asyncio.sleep(0.5)
  links = driver.find_elements(By.TAG_NAME, "a")
  facebook_links = [link.get_attribute("href") for link in links if link.get_attribute("href") and 'facebook.com' in link.get_attribute("href")]
  driver.quit()
  filtered_facebook_links = list(filter(lambda x: x != None, (get_facebook_links(l, fb_page) for l in facebook_links)))
  saveToCSV(filtered_facebook_links, f"{fb_page}.csv")

loop = asyncio.get_event_loop()


loop.create_task(main())

<Task pending name='Task-15' coro=<main() running at C:\Users\canet_f7h5kzg\AppData\Local\Temp\ipykernel_13940\1250819090.py:32>>

In [10]:
import requests
from bs4 import BeautifulSoup

# The URL of the Facebook page (ensure it's publicly accessible)
url = "https://www.facebook.com/fopsco.ph"

# Define headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

try:
    # Send a GET request to the URL
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all links in the page
    links = soup.find_all('a', href=True)

    # Filter and print Facebook links
    facebook_links = [link['href'] for link in links if 'facebook.com' in link['href']]
    for fb_link in facebook_links:
        print(fb_link)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")


An error occurred: 400 Client Error: Bad Request for url: https://www.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2Ffopsco.ph
